In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import xlrd

In [2]:
df = pd.read_parquet('little_dental.pq')

In [5]:
df.memory_usage()/(1024*1024)

Index           0.000122
ACTIVITYID     25.261772
PGID            3.157722
OID             6.315443
RPID           12.630886
PATID          12.630886
USERID         25.261772
SESSIONID      25.261772
WEBSERVERIP    25.261772
REMOTEIP       25.261772
PAGENAME       25.261772
CREATEDON      25.261772
dtype: float64

In [6]:
df['timestamp'] = pd.to_datetime(df.CREATEDON)
del(df['PGID'])
del(df['OID'])
del(df['CREATEDON'])
del(df['WEBSERVERIP'])
del(df['RPID'])
del(df['PATID'])
df.head(2)

,ACTIVITYID,RPID,PATID,USERID,SESSIONID,REMOTEIP,PAGENAME,timestamp
0,124130650925,41019304,41052965,RAYLENERAYFORD2021,bvh455ixumcrgc553pr0ul45,23.30.205.249,SearchPatients,2021-11-17 04:32:36.943
1,124130650935,41019304,41052965,RAYLENERAYFORD2021,bvh455ixumcrgc553pr0ul45,23.30.205.249,SearchPatients,2021-11-17 04:32:39.860


In [10]:
df.memory_usage()/(1024*1024)

Index          0.000122
ACTIVITYID    25.261772
USERID        25.261772
SESSIONID     25.261772
REMOTEIP      25.261772
PAGENAME      25.261772
timestamp     25.261772
dtype: float64

In [14]:
df.to_parquet('little_dental_reduced_mem.pq')

In [2]:
df = pd.read_parquet('little_dental_reduced_mem.pq')

In [3]:
sessiondf = pd.DataFrame(columns=['user', 'nop', 'dur_in_min', 'start', 'end'])
for k, gbdf in df.groupby(['USERID', 'SESSIONID']):
    user, session = k
    number_of_pages = gbdf.PAGENAME.count()
    sessiondf.loc[session, 'user'] = user
    sessiondf.loc[session, 'nop'] = number_of_pages
    duration = max(gbdf.timestamp) - min(gbdf.timestamp)
    sessiondf.loc[session, 'dur_in_min'] = duration.seconds/60.0
    sessiondf.loc[session, 'start'] = min(gbdf.timestamp).timestamp()
    sessiondf.loc[session, 'end'] = max(gbdf.timestamp).timestamp()
sessiondf.head()

,user,nop,dur_in_min,start,end
1e0athmc3knhdp451csz4on4,KIMBARKSDALE2021,419,479.083,1.63894e+09,1.63906e+09
1vq3hf55noreo4qizcmyh2jc,ABARCAESPIRITU2021,40,575.417,1.63963e+09,1.63967e+09
222nip552t4drw45wmmybvzv,ROBINSHERRILL2021,50,474.567,1.63998e+09,1.64001e+09
230jjez1aurv2sjomvqq5h45,ABARCAESPIRITU2021,2,0.133333,1.64011e+09,1.64011e+09
2yeye1eyt31wa43mlm0a2dir,ROBINSHERRILL2021,15,187.867,1.63938e+09,1.63948e+09


In [4]:
df.memory_usage(deep=True).sum()

1248373541

In [5]:
sessiondf.to_parquet('dental_sessions.pq')

In [6]:
sessiondf.memory_usage(deep=True).sum()

46119802